last update: 2016-8-17

### 练习目标
本练习的目标为：
1. 将csv文件中的数据导入MongoDB
2. 提供一个Web站点对这些数据进行查询

### 练习数据
本练习数据采用了Gourplens最新的（2016 Jan）发布的基于240,000名观众针对33,000部电影所做出的22,000,000次评分数据进行，数据文件大小为114MB。该可以通过访问如下链接进行获取：
http://grouplens.org/datasets/movielens/
<p>练习所用的文件包含如下四个：
<li>movies.csv - 含所有影片的基础信息</li>
<li>link.csv - 含所有影片多对应的IMDBID，TMDBID</li>
<li>ratings.csv - 含所有影片的评分信息</li>
<li>tags.csv - 含所有影片的评论信息</li>

In [3]:
#该语句表示需要Jupyter获取matplotlib生成的所有图形，否则将无法在notbook中显示
%matplotlib inline 

#导入相应的库
import numpy as np

import pandas as pd
from pandas import DataFrame, Series

import matplotlib.pyplot as plt
import json

import pymongo
from pymongo import MongoClient

#Connect to MongoDB on Azure
#conn = MongoClient("mongodb://illusion615:UmgIQuFqR7Hg5gDbVa3hAngEEuvs2XTlXJzJytSMqfIxtrPWtXhEKpaV2zggeeG15tdy3sQ7E9XjAfQowygPwg==@illusion615.documents.azure.com:10250/?ssl=true")
conn = MongoClient("mongodb://localhost")
db = conn['MovieReserch']
coll = db['Movies']

Database(MongoClient('localhost', 27017), 'MovieReserch')

In [ ]:
path_Movie = 'C:/Users/wellszha/Downloads/Python Practice/ml-latest/movies.csv'
path_Links = 'C:/Users/wellszha/Downloads/Python Practice/ml-latest/links.csv'
path_Tags = 'C:/Users/wellszha/Downloads/Python Practice/ml-latest/tags.csv'
path_Rating ='C:/Users/wellszha/Downloads/Python Practice/ml-latest/ratings.csv'

#处理movies以及link表
dfMovie = DataFrame(pd.read_csv(path_Movie))
dfLinks = DataFrame(pd.read_csv(path_Links))
dfRatings = pd.read_csv(path_Rating)
dfTags = pd.read_csv(path_Tags)

### Insert
MongoDB主要采用两个命令进行数据的插入<p>
insert_one()和insert_many()<p>
所带的参数是类似于dictionary一样的结构，例如：<p>
<pre><code>
{
    'City':'Shanghai',
    'Province':'Shanghai',
    'Country':'China',
    'Zipcode':[200100,200110,200112,200113]
}
</code></pre>

### 需要注意的点

#### 数据转换
由于MongoDB采用BSON存储数据，而integer在MongoDB中是不支持的（需要继续研究确认），所以都需要转换成字符串。例如：<p>
<pre><code>
str(dataFrame.iloc[1]['userId'])
</code></pre>





In [ ]:
#merge movies and link by movieId
dfMergedMovie=pd.merge(dfMovie,dfLinks)

#build the ds
for i in range(0,len(dfMergedMovie)):
    #Define the structure of dataset to insert into MongoDB
    ds = {
        "movieId":"",
        "title":"",
        "genres":[],
        "year":"",
        "imdbId":"",
        "tmdbId":"",
        "rating":[],
        "tags":[]
    }
    ds["movieId"]= str(dfMergedMovie.loc[i]["movieId"])
    ds["title"] = dfMergedMovie.loc[i]["title"]
    #Check if movie exists in database
    if coll.find({"movieId":ds["movieId"]}).count() == 0:
        genres = []
        ds["genres"] = dfMergedMovie.loc[i]["genres"].split("|")
        ds["imdbId"] = str(dfMergedMovie.loc[i]["imdbId"])
        ds["tmdbId"] = dfMergedMovie.loc[i]["tmdbId"]

        dfR = dfRatings[dfRatings["movieId"]==dfMergedMovie.loc[i]["movieId"]]
        for j in range(0,len(dfR)):
            r = {"userId":"","rating":0,"timestamp":""}
            r["userId"] = str(dfR.iloc[j]["userId"])
            r["rating"] = dfR.iloc[j]["rating"]
            r["timestamp"] = str(dfR.iloc[j]["timestamp"])
            ds["rating"].append(r)

        dfT = dfTags[dfTags['movieId']==dfMergedMovie.loc[i]["movieId"]]
        for h in range(0,len(dfT)):
            t = {"userId":"","tag":"","timestamp":""}
            t["userId"] = str(dfT.iloc[h]["userId"])
            t["tag"] = dfT.iloc[h]["tag"]
            t["timestamp"] = str(dfT.iloc[h]["timestamp"])
            ds["tags"].append(t)
        coll.insert_one(ds)
        print("Movie %s insert completed, overall progress %s of %s" 
              %(ds["title"],i,len(dfMergedMovie)))
    else:
        print("Bypassing existing %s, overall progress %s of %s"
             %(ds["title"],i,len(dfMergedMovie)))

Bypassing existing Toy Story (1995), overall progress 0 of 34208
Bypassing existing Jumanji (1995), overall progress 1 of 34208
Bypassing existing Grumpier Old Men (1995), overall progress 2 of 34208
Bypassing existing Waiting to Exhale (1995), overall progress 3 of 34208
Bypassing existing Father of the Bride Part II (1995), overall progress 4 of 34208
Bypassing existing Heat (1995), overall progress 5 of 34208
Bypassing existing Sabrina (1995), overall progress 6 of 34208
Bypassing existing Tom and Huck (1995), overall progress 7 of 34208
Bypassing existing Sudden Death (1995), overall progress 8 of 34208
Bypassing existing GoldenEye (1995), overall progress 9 of 34208
Bypassing existing American President, The (1995), overall progress 10 of 34208
Bypassing existing Dracula: Dead and Loving It (1995), overall progress 11 of 34208
Bypassing existing Balto (1995), overall progress 12 of 34208
Bypassing existing Nixon (1995), overall progress 13 of 34208
Bypassing existing Cutthroat Isl

### 查询


In [148]:
print(coll.find({'title':'Backbeat (1993)'})['imdbId'])

TypeError: index 'imdbId' cannot be applied to Cursor instances